Итак, мы взялись исследовать криптовалюты. Для начала краткий дайвинг в получение и работу с данными по криптовалютам разных бирж. Основной источник:
https://blog.patricktriest.com/analyzing-cryptocurrencies-python/

In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

In [ ]:
def get_quandl_data(quandl_id):
    '''Download and cache Quandl dataseries'''
    cache_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(quandl_id, cache_path))
    return df


In [ ]:
# Pull Kraken BTC price exchange data
btc_usd_price_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

In [ ]:
btc_usd_price_kraken.head()

In [ ]:
btc_usd_price_kraken.tail()['Open']

In [ ]:
btc_usd_price_kraken.tail()['Open'][0]

In [ ]:
btc_usd_price_kraken.tail()['Open'].mean()

In [ ]:
btc_usd_price_kraken.resample('W')['Open'].mean()

In [ ]:
btc_usd_price_kraken.tail()['Open'].mean()

In [ ]:
weak = btc_usd_price_kraken.resample('W')

In [ ]:
trace = go.Scatter(x=btc_usd_price_kraken.resample('W')['Open'].mean().index,y=btc_usd_price_kraken.resample('W')['Open'].mean())

In [ ]:
py.iplot([trace])

Хорошо, с данными уже можно работать. Попробуем получить БПФ от сигнала котировок биткойна. Для начала разберёмся со scipy чтобы считать БПФ...

In [ ]:
import scipy

In [ ]:
scipy.fft(btc_usd_price_kraken['Weighted Price'])

In [ ]:
fft_data = scipy.fft(btc_usd_price_kraken.loc['2017-Aug':'2018-Aug','Weighted Price'])

In [ ]:
print (fft_data)

In [ ]:
absd = abs(fft_data)

In [ ]:
py.iplot([go.Scatter(y=absd)])

In [ ]:
from numpy import *

In [ ]:
x = [math.sin(2*math.pi*4*i/256) for i in range(0,256)]

In [ ]:
data = scipy.fft(x)

In [ ]:
py.iplot([go.Scatter(y=abs(data[:-int(data.size/2)]))])

Окей, всё понятно, получили заветную синусоиду, с нормировкой разберёмся далее.
А пока используем данные для частотного анализа по бирже Kraken...

In [ ]:
data = btc_usd_price_kraken.loc['2013-Aug':'2018-Aug','Weighted Price']

In [ ]:
from numpy import *

In [ ]:
wSize=32
wSize2 = int(wSize/2)
freqMaxIdx=8

# расчёт периодов сигналов котировок
fs=1 #1day inverse
f0=fs/wSize
deltaF=fs/(2*wSize)

frame = data[0:wSize]
window = blackman(wSize)
m = array([abs(np.fft.fft(frame*window)/wSize)[:freqMaxIdx]])
d = [data.index[wSize]]
summ = []
summ.append(sum(m[0]))

# здесь он продолжается...
per = []
for j in range(0,wSize):
     per.append(str(int(floor(1/(f0+j*deltaF))))+'​') #dirty hack U+200B invisible word delimeter =) plotly вредный, перемасштабирует график, а это нам не нужно

In [ ]:
for i in range(1,data.size-wSize):
    frame = data[i:i+wSize]
    window =blackman(wSize) # окно Блэкмана
    m = append(m,[abs(np.fft.fft(frame*window)/wSize)[:freqMaxIdx]],0) #использую numpy ибо значения нормированные возращает в отличие от scipy
    summ.append(sum(m[i]))
    d.append(data.index[i+wSize])

In [ ]:
heatmap = go.Heatmap(
    z=m.transpose(),
    x=d,
    y=per,)

In [ ]:
layout = go.Layout(
    title='Fourier decomposition of BTC quotes (USD) [Kraken]',
    titlefont=dict(
        family='Courier New, monospace',
        size=22,
        color='#7f7f7f'
    ),
    yaxis=dict(
        title='Period, days',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
    ),
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
    ),
)
fig = go.Figure(data=[heatmap], layout=layout)


In [ ]:
py.iplot(fig)

Вот здесь по-подробнее, ибо в этом графике вся соль и суть работы. Мы имеем разложение по гармоникам графика курса биткойна на бирже Kraken. Видно, что основные колебания происходят периодом около месяца, и что интересно, возможно намечается новая волна на повышение курса (правый нижний угол). Все остальные гармоники имеют меньшие амплитуды, но в тоже время можно увидеть появление и исчезание колебаний на нижних частотах. Если делать преобразование более точным по частоте, то вырисовываются более чёткие частотные линии. К ним я ещё вернусь, но их нужно верифицировать. Неясно, являются ли они реальными или это следствие принципа неопределённости. Хорошо бы было бы увеличить точность по частоте и по времени одновременно, но для этого нужны более точные исходные данные. Нужно будет заняться этим вопросом. Пока резюме - нужны дополнительные исследования связи появления гармоник с новостями, а также расширение исследований на другие валюты и биржи.

Идея: основная задача - научиться прогнозировать частоты, тогда можно создать поличастотного бота иили вебсервис по продаже советов.

In [ ]:
py.iplot([go.Scatter(y=summ)])

Окей, видно, что сигнал оконный хорошо сглажен окном Блэкмана и обрабатываем мы не лажу.

In [ ]:
py.iplot([go.Scatter(x=data.index,y=data)])

In [ ]:
w=blackman(512)

In [ ]:
np.absolute(1 + 1j)

In [ ]:
np.absolute(2 + 0j)

In [ ]:
abs(0+ 2j)

Хаха, модуль работает правильно!

Теперь мы хотим получить VAR величину и закоррелировать её с хэшрейтами для опеределения влияния хэшрейтов на величину VAR.

(https://ru.wikipedia.org/wiki/Value_At_Risk)


Получим данные по иным криптовалютам с биржи Poloniex.

In [ ]:
def get_json_data(json_url, cache_path):
    '''Download and cache JSON data, return as a dataframe.'''
    try:        
        f = open(cache_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(json_url))
    except (OSError, IOError) as e:
        print('Downloading {}'.format(json_url))
        df = pd.read_json(json_url)
        df.to_pickle(cache_path)
        print('Cached {} at {}'.format(json_url, cache_path))
    return df

base_polo_url = 'https://poloniex.com/public?command=returnChartData&currencyPair={}&start={}&end={}&period={}'
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # get data from the start of 2015
end_date = datetime.now() # up until today
pediod = 86400 # pull daily data (86,400 seconds per day)

def get_crypto_data(poloniex_pair):
    '''Retrieve cryptocurrency data from poloniex'''
    json_url = base_polo_url.format(poloniex_pair, start_date.timestamp(), end_date.timestamp(), pediod)
    data_df = get_json_data(json_url, poloniex_pair)
    data_df = data_df.set_index('date')
    return data_df

altcoins = ['ETH','LTC','XRP','ETC','STR','DASH','SC','XMR','XEM']

altcoin_data = {}
for altcoin in altcoins:
    coinpair = 'BTC_{}'.format(altcoin)
    crypto_price_df = get_crypto_data(coinpair)
    altcoin_data[altcoin] = crypto_price_df

In [ ]:
altcoin_data['ETH'].tail()

Здесь цены выражены в биткойнах.

Получим курсы BTC с других криптобирж, а потом усредним и получим цены ны альткойны.

In [ ]:
# Pull pricing data for 3 more BTC exchanges
exchanges = ['COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

exchange_data['KRAKEN'] = btc_usd_price_kraken

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df
    
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col]
    return pd.DataFrame(series_dict)

# Merge the BTC price dataseries' into a single dataframe
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

# Calculate the average BTC price as a new column
btc_usd_datasets['avg_btc_price_usd'] = btc_usd_datasets.mean(axis=1)

# Calculate USD Price as a new column in each altcoin dataframe
for altcoin in altcoin_data.keys():
    altcoin_data[altcoin]['price_usd'] =  altcoin_data[altcoin]['weightedAverage'] * btc_usd_datasets['avg_btc_price_usd']

Дальше повторяю действия из статьи, чтобы посмотреть на совместные колебания курсов криптовалют. Очень интересно: последний год колебания курсов сильно коррелируют.

In [ ]:
# Merge USD price of each altcoin into single dataframe 
combined_df = merge_dfs_on_column(list(altcoin_data.values()), list(altcoin_data.keys()), 'price_usd')
# Add BTC price to the dataframe
combined_df['BTC'] = btc_usd_datasets['avg_btc_price_usd']
# Calculate the pearson correlation coefficients for cryptocurrencies in 2016
combined_df_2018 = combined_df[combined_df.index.year == 2018]
combined_df_2018.pct_change().corr(method='pearson')

In [ ]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
        
    # Form Trace For Each Series
    trace_arr = []
    for index, series in enumerate(series_arr):
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

# Chart all of the altocoin prices
df_scatter(combined_df, 'Cryptocurrency Prices (USD)', seperate_y_axis=False, y_axis_label='Coin Value (USD)', scale='log')

Рассмотрим простейший случай равнодолевого портфеля из койнов разных видов. Посчитаем величину VaR для такого портфеля.

In [ ]:
print(altcoins)

In [ ]:
stocks = altcoins.copy()
stocks.append('BTC')
print(stocks)
numAssets = len(stocks)

# доли активов в пакете
weights = []
for i in range(0,numAssets):
    weights.append(1/numAssets)
print(weights,"\n")

# данные по цене койнов за большой период времени
data = combined_df.dropna()
#data = data.loc['2018-2-1':'2018-8-1']
# стоимости активов
V = (data * weights).transpose().mean()
print("Ininitial Inverstments ", round(sum(V[0]),1),"$")

# колебания стоимости портфеля за период
dailyReturn = V - V.shift(1)

m = dailyReturn.mean()
d = sqrt(dailyReturn.var())
alpha = 3.715 # --> 0.01% случаев
VaR = -(dailyReturn.mean() - alpha * d) # такой риск можно получить
print("Mean Return", m)
print("Return Disp", d)
print("VAR ", VaR) # в долларах

In [ ]:
import seaborn as sns

sns.distplot(dailyReturn.dropna())

In [ ]:
# sort the data:
data_sorted = np.sort(dailyReturn)

# calculate the proportional values of samples
p = 1. * arange(len(dailyReturn)) / (len(dailyReturn) - 1)

f = lambda x: np.interp(x, data_sorted, p)

profit_prob = 100* (1 - f(0))
risk = (1 - f(VaR)) * 100
print("expected_profit = ", round(m,1),"$")
print("profit > 0: ", round(profit_prob,1),"%")
print("VaR > ", round(VaR,1), "$: ", round(risk,1),"%") # более точное значение вероятности риска
print("Ininitial Inverstments ", round(sum(V[0]),1),"$")
print("Profit Persent ", round(100*m/sum(V[0]),1),"%")

Все эти величины показывают, сколько прибыли или убытков можно получить с вложений в портфель заданной стоимости. К примеру банковский процент по вкладам альфа-банка сейчас 2.5% при вложении денег на год, что более выгодно и надёжно, т.к. защищено страховыми обязательствами в случае банкротства банка. Но при этом например нельзя снимать деньги в течении года. Риски кажутся весьма высокими... Допустим ситуацию, когда мы имеем прибыль в течении 100 дней получаем..

In [ ]:
print(100*round(m,1),"$")

Весьма вероятна ситуация, когда мы можем на следующий день потерять величину большую VaR и убытки могут составить более чем:

In [ ]:
print(round(VaR - 100 * m,1),"$")

или

In [ ]:
print(round(100*round(VaR - 100 * m,1)/round(sum(V[0]),1),1),"%")

От первоначальных вложений... В принципе не так уж и опасно, но люди пишут, по идее нужно правило 10 * VaR (очень сомневаюсь что это правило действительно используется в наших реалиях).

In [ ]:
stocks = altcoins.copy()
stocks.append('BTC')
numAssets = len(stocks)



def get_VaR(cointypes, weights, combined_df):
    # данные по цене койнов за большой период времени
    data = combined_df.dropna()
    #data = data.loc['2018-2-1':'2018-8-1']
    # стоимости активов
    V = (data * weights).transpose().mean()

    # колебания стоимости портфеля за период
    dailyReturn = V - V.shift(1)

    m = dailyReturn.mean()
    d = sqrt(dailyReturn.var())
    alpha = 3.715 # --> 0.01% случаев
    VaR = -(dailyReturn.mean() - alpha * d) # такой риск можно получить
    return VaR


for i in range(0,numAssets):
# доли активов в пакете
    weights = []
    for j in range(0,numAssets):
        value = 0
        if(i == j):
            value = 1
        weights.append(value)
    print(stocks[i],round(get_VaR(stocks, weights, combined_df),2))

In [ ]:
VaRs = [["ETH", 10.42],
["LTC", 2.83],
["XRP", 0.02],
["ETC", 0.47],
["STR", 0.01],
["DASH", 10.22],
["SC", 0.0],
["XMR", 4.04],
["XEM", 0.01],
["BTC", 126.8]]



XEM - нет совсем
XRP - нет совсем
LTC - мало инфы

In [ ]:
import csv

f = open('coin_herfindal.csv', 'r')

raws = []
with f:
    reader = csv.reader(f)
    for row in reader:
        raws.append(row)
            
currs = []
varss = []
herfs = []
for i in range(0,len(VaRs)):
    for j in range(0,len(raws)):
        if(raws[j][0] == VaRs[i][0]):
            currs.append(VaRs[i][0])
            varss.append(float(VaRs[i][1]))
            herfs.append(float(raws[j][1]))
            break
            
            
def mean(someList):
    total = 0
    for a in someList:
        total += float(a)
    mean = total/len(someList)
    return mean
def standDev(someList):
    listMean = mean(someList)
    dev = 0.0
    for i in range(len(someList)):
        dev += (someList[i]-listMean)**2
    dev = dev**(1/2.0)
    return dev
def correlCo(someList1, someList2):

    # First establish the means and standard deviations for both lists.
    xMean = mean(someList1)
    yMean = mean(someList2)
    xStandDev = standDev(someList1)
    yStandDev = standDev(someList2)
    # r numerator
    rNum = 0.0
    for i in range(len(someList1)):
        rNum += (someList1[i]-xMean)*(someList2[i]-yMean)

    # r denominator
    rDen = xStandDev * yStandDev

    r =  rNum/rDen
    return r


print(currs)
print(correlCo(varss,herfs))